# Neurocombat TOP versus StrokeMRI

Note this must be run in the `neurocombaty` environment

## import libraries

In [ ]:
from neuroCombat import neuroCombat
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
needs to be replaced with more current sets

In [ ]:
our_mri_data = pd.read_csv('C:/Projects/brainspin/not_pushed/data_anonymized/assembled/StrokeMRI_stitched.csv')
our_top_data = pd.read_csv('C:/Projects/brainspin/not_pushed/data_anonymized/assembled/top_stitched.csv')

In [ ]:
our_mri_data

In [ ]:
# mri_transposed = our_mri_data.T
# mri_transposed.shape

In [ ]:
# top_transposed = our_top_data.T
# top_transposed.shape

We only need a set of common columns, so now we must reshape to get common columns. We will have to begin by renaming columns the same thing.

In [ ]:
#shared_columns

In [ ]:
our_mri_data.columns

In [ ]:
# for column in our_top_data.columns:
#     print(column)

In [ ]:
our_top_data['GM_vol'] = our_top_data['GM_vol_Liter']
our_top_data['WM_vol'] = our_top_data['WM_vol_Liter']
our_top_data['CSF_vol'] = our_top_data['CSF_vol_Liter']
our_top_data['GM_ICVRatio'] = our_top_data['GM_ICVRatio_ratio GM/ICV'] 
our_top_data['WMH_vol'] = our_top_data['GMWM_ICVRatio_ratio (GM+WM)/ICV']
our_top_data['WMH_count'] = our_top_data['WMH_count_n lesions (integer)']
our_top_data['DeepWM_B'] = our_top_data['DeepWM_B_SD/mean']
our_top_data['DeepWM_L'] = our_top_data['DeepWM_L_SD/mean']
our_top_data['DeepWM_L'] = our_top_data['DeepWM_R_SD/mean']
our_top_data['ACA_B']= our_top_data['ACA_B_SD/mean']    
our_top_data['ACA_L']= our_top_data['ACA_L_SD/mean']           
our_top_data['ACA_R']= our_top_data['ACA_R_SD/mean']            
our_top_data['MCA_B']= our_top_data['MCA_B_SD/mean']            

In [ ]:
shared_columns = (
        our_mri_data.columns.intersection(our_top_data.columns)).to_list()

In [ ]:
shared_columns

In [ ]:
top_selection = our_top_data[shared_columns].drop(['Unnamed: 0','renumber'],axis=1)
mri_selection = our_mri_data[shared_columns].drop(['Unnamed: 0','renumber'],axis=1)


In [ ]:
top_selection = top_selection.dropna(axis=0)
mri_selection = mri_selection.dropna(axis=0)

In [ ]:
top_selection = top_selection.T
mri_selection = mri_selection.T
#mri_selection

In [ ]:
mri_selection

In [ ]:
top_selection = top_selection.rename(columns={x:y for x,y in zip(top_selection.columns,range(len(mri_selection.columns),(len(top_selection.columns)+ len(mri_selection.columns))))})

In [ ]:
both_together = pd.concat([ mri_selection, top_selection], axis=1, join="inner")
both_together

In [ ]:
# make sure there are no NaNs
both_together.isna().sum().sum()

In [ ]:
# save off csv
both_together.to_csv('both_together.csv')

In [ ]:
# make and save of csv of features only
features_only = both_together[:12]
features_only.to_csv('features_only.csv')

In [ ]:
features_only.T.columns

So that will map to 0,1,2,3,4,5,6,7,8,9,10,11,

In [ ]:
# neurocombat hates NaNs in the data, let's impute them , or better replace them with 1000

In [ ]:
ft = features_only.reset_index()
ft = ft.rename(columns={"index": "A"})
ft = ft.drop(['A'], axis=1)
ft = ft.dropna()

In [ ]:
bt= both_together.reset_index()
bt = bt.rename(columns={"index": "A"})

bt = bt.drop(['A'], axis=1)
bt = bt.dropna()

In [ ]:
bt

In [ ]:
ft

In [ ]:
ft.to_csv('ft.csv')

In [ ]:
data = np.genfromtxt('ft.csv', delimiter=",", skip_header=1)
data

In [ ]:
# array_from_both_together = both_together.to_numpy()
# data = array_from_both_together

In [ ]:
data = data[:, 1:]

In [ ]:
data.shape

In [ ]:
data

In [ ]:
# Specifying the batch (scanner variable) as well as a sex covariate to preserve:
first_columns_as_one = [1] * len(mri_selection.columns)
last_columns_as_two = [2] * len(top_selection.columns)
covars = {'Batch':first_columns_as_one + last_columns_as_two,
          'Sex':both_together.loc['Sex',:].values.tolist(),
           'Age':both_together.loc['Age',:].values.tolist(),} 
covars = pd.DataFrame(covars) 

In [ ]:
#covars['Sex'] = covars['Sex'] + 1

In [ ]:
covars

In [ ]:
# specifify sex as categorical
categorical_cols = ['Sex']
# To specify the name of the variable that encodes for the scanner/batch covariate:
batch_col_mine = 'Batch'

#Harmonization step:
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col_mine,
    categorical_cols=categorical_cols)["data"]

In [ ]:
data_combat

In [ ]:
data

In [ ]:
data - data_combat

### So that is the absolute difference, but let;s look at a percent

In [ ]:
changes_in_percent = (data - data_combat)/data

In [ ]:
tweak = pd.DataFrame(changes_in_percent)
tweak

In [ ]:
tweak_describe = tweak.describe()

In [ ]:
maxes = tweak_describe.tail(1)

In [ ]:
maxes.T.describe()

In [ ]:
# plt.imshow(tweak, cmap ="RdYlBu")
# plt.colorbar()
  

In [ ]:
print(type(both_together.T.columns.tolist()))

So essentially neurocombat makes a tiny tweak in the data, but the tweak is so small, we could say it recognizes there is no need for remapping the data...it would only change at max about 4% to any value, and in most cases it would change 1 or less percent.

# SUPER IMPORTANT: this notebook is not done. It does not have the ASL values. Before this cane be done data should be mapped into the ideal format, then we can easily have all thecorrect common columsn!

In [ ]:
# rename combat_data
data_combat_panda = pd.DataFrame(data_combat)
data_combat_mri= data_combat_panda.loc[ :, :515]
data_combat_top= data_combat_panda.loc[ :, 516:]
data_combat_mri['type'] = features_only.T.columns.tolist()
data_combat_top['type'] = features_only.T.columns.tolist()

In [ ]:

data_combat_mri

In [ ]:
data_combat_top

In [ ]:
for df in [data_combat_top,data_combat_mri]:
    # shift column 'type' to first position
    first_column = df.pop('type')
    # insert column using insert(position,column_name,first_column) function
    df.insert(0, 'type', first_column)
    

In [ ]:
data_combat_top

In [ ]:
top_selection

In [ ]:
# violin plot of TOP
complete_harmonised_nonharmonised_merged = complete_harmonised.append(complete_nonharmonised)

sns.set_style("whitegrid")
sns.catplot(data = complete_harmonised_nonharmonised_merged, x = 'Harmonisation', y = "TotalGM_B_CBF", hue ="Sex", 
split=True, inner = 'quartile', kind = 'violin', 
col = 'Site', height = 5, aspect = 0.6)
plt.ylim((-20,150))
plt.show()